<a href="https://colab.research.google.com/github/JetteKA/BigData/blob/master/cnn_retrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sklearn.metrics as metrics
from sklearn.cluster import KMeans
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D


Using TensorFlow backend.


In [0]:
n_classes = 10

epochs = 15
batch_size = 50

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

In [0]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


4423680/4422102 [==============================] - 2s 0us/step


In [0]:
def label_manually(n, x_train, y_train):
    '''Human powered labeling (actually copying from the prelabeled MNIST dataset).'''

    x_labelled = x_train[:n]
    y_labelled = y_train[:n]

    return x_labelled, y_labelled

In [0]:
def reshape_x_for_cnn(data, img_rows=img_rows, img_cols=img_cols):


  data = data.reshape(data.shape[0], img_rows, img_cols, 1)
  data = data.astype('float32')
  data /=255

  return data

def reshape_y_for_cnn(data, n_classes=n_classes):
  return keras.utils.to_categorical(data, n_classes)


In [0]:
x_test = reshape_x_for_cnn(x_test)

y_test = reshape_y_for_cnn(y_test)

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
train_acc = []
test_acc = []
train_loss = []
test_loss = []
num_labels = []

In [0]:
def train(model, x_labelled, y_labelled):


  x_label_cnn = reshape_x_for_cnn(x_labelled)


  y_label_cnn = reshape_y_for_cnn(y_labelled)

  model.fit(x_label_cnn, y_label_cnn,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1)
  test_score = model.evaluate(x_test, y_test, verbose=0)
  print('Test loss:', test_score[0])
  print('Test accuracy:', test_score[1])
  test_loss.append(test_score[0])
  test_acc.append(test_score[1])

  train_score = model.evaluate(x_label_cnn, y_label_cnn, verbose=0)
  print('Train loss:', train_score[0])
  print('Train accuracy:', train_score[1])
  train_loss.append(train_score[0])
  train_acc.append(train_score[1])

  num_labels.append(x_labelled.shape[0])

  
  return model


In [0]:
for x in range(50, 10050, 50):
    print(x)
    x_labelled, y_labelled = label_manually(x, x_train, y_train)
    model = train(model, x_labelled, y_labelled)

50
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/15





50/50 [==============================] - 12s 238ms/step - loss: 2.3047 - acc: 0.0800
Epoch 2/15
50/50 [==============================] - 0s 318us/step - loss: 2.2068 - acc: 0.2600
Epoch 3/15
50/50 [==============================] - 0s 321us/step - loss: 2.0656 - acc: 0.2600
Epoch 4/15
50/50 [==============================] - 0s 323us/step - loss: 1.9618 - acc: 0.4600
Epoch 5/15
50/50 [==============================] - 0s 300us/step - loss: 1.8114 - acc: 0.3400
Epoch 6/15
50/50 [==============================] - 0s 317us/step - loss: 1.6975 - acc: 0.3800
Epoch 7/15
50/50 [==============================] - 0s 316us/step - loss: 1.7560 - acc: 0.4000
Epoch 8/15
50/50 [==============================] - 0s 328us/step - loss: 1.6213 - acc: 0.4200
Epoch 9/15
50/50 [==============================] - 0s 348us/step - loss: 1.6288 - acc: 0.4800
Epoch 10/15
50/50 [====================

In [0]:
plt.plot(num_labels, test_acc, label="Testset")
plt.plot(num_labels, train_acc, label="Trainset")
plt.xlabel('Number of labelled images')
plt.ylabel("Accuracy")
plt.axis(ymin=0, ymax=1.01, xmin=0)
plt.legend()
plt.savefig("drive/My Drive/BD/CNN_retrain")

In [0]:

MyFile=open('drive/My Drive/BD/CNN_retrain_acc.txt','w')

for element in test_acc:
     MyFile.write(str(element))
     MyFile.write('\n')
MyFile.close()

In [0]:

MyFile=open('drive/My Drive/BD/CNN_retrain_acc_train.txt','w')

for element in train_acc:
     MyFile.write(str(element))
     MyFile.write('\n')
MyFile.close()

In [0]:

MyFile=open('drive/My Drive/BD/CNN_retrain_num_labels.txt','w')

for element in num_labels:
     MyFile.write(str(element))
     MyFile.write('\n')
MyFile.close()

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='model.png',show_shapes=True)

In [0]:
# history = model.fit(x_label_cnn, y_label_cnn,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [0]:
# # Plot training & validation accuracy values
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()